In [1]:
# Parameters

# Folder on local machine where to create the output and temporary folders
output_path = "/Users/pedroszekely/Downloads/kypher"

# The names of the output and temporary folders
output_folder = "wikidata_os_v5"
temp_folder = "temp.wikidata_os_v5"

# The location of input Wikidata files
wikidata_folder = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20200803-v4/"
wikidata_folder = "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/"
# The wikidata_os files can be downloaded from https://drive.google.com/drive/folders/1V6oAQKmwQ4LJnrBai-uv5gHWphFSCt50?usp=sharing

# Location of the cache database for kypher
cache_path = "/Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v4"

# Whether to delete the cache database
delete_database = False

# shortcuts to commands
kgtk = "time kgtk --debug"
# kgtk = "kgtk --debug"

# KGTK Tutorial

Beer sites:
- https://www.realbeer.com/edu/health/calories.php
- http://getdrunknotfat.com/alcohol-content-of-beer/

In [2]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

import altair as alt

import papermill as pm

In [3]:
# The names of files in the KGTK Wikidata distirbution that we will use in this notebook.
file_names = {
    "claims": "claims.tsv.gz",
    "label": "labels.en.tsv.gz",
    "alias": "aliases.en.tsv.gz",
    "description": "descriptions.en.tsv.gz",
    "item": "claims.wikibase-item.tsv.gz",
    "qualifiers": "qualifiers.tsv.gz",
    "sitelinks": "sitelinks.tsv.gz",
    "qualifiers_time": "qualifiers.time.tsv.gz",
    "property_datatypes": "metadata.property.datatypes.tsv.gz",
    "isa": "derived.isa.tsv.gz",
    "p279star": "derived.P279star.tsv.gz",
    "p279": "derived.P279.tsv.gz"
}

# We will define environment variables to hold the full paths to the files as we will use them in the shell commands
kgtk_environment_variables = []

os.environ['WIKIDATA'] = wikidata_folder
kgtk_environment_variables.append('WIKIDATA')

for key, value in file_names.items():
    variable = key.upper()
    os.environ[variable] = wikidata_folder + value
    kgtk_environment_variables.append(variable)
    
# KGTK creates a SQLite database to index the knowledge graph.
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)
kgtk_environment_variables.append('STORE')

# We will create many temporary files, so set up a folder for outputs and one for the temporary files.
os.environ['TEMP'] = "{}/{}".format(output_path, temp_folder) 
os.environ['OUT'] = "{}/{}".format(output_path, output_folder) 
kgtk_environment_variables.append('TEMP')
kgtk_environment_variables.append('OUT')

# Envronment variables with shortcuts to the commands we use often
os.environ['kgtk'] = kgtk
os.environ['kypher'] = "time kgtk --debug query --graph-cache " + os.environ['STORE']
os.environ['kypher'] = "time kgtk query --graph-cache " + os.environ['STORE']
kgtk_environment_variables.append('kgtk')
kgtk_environment_variables.append('kypher')

# We'll save the current working directory so we can call into other example notebooks later
os.environ["EXAMPLES_DIR"] = os.getcwd()
kgtk_environment_variables.append('EXAMPLES_DIR')

kgtk_environment_variables.sort()
for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

ALIAS: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/aliases.en.tsv.gz"
CLAIMS: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/claims.tsv.gz"
DESCRIPTION: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/descriptions.en.tsv.gz"
EXAMPLES_DIR: "/Users/pedroszekely/Documents/GitHub/kgtk/examples"
ISA: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/derived.isa.tsv.gz"
ITEM: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/claims.wikibase-item.tsv.gz"
LABEL: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/labels.en.tsv.gz"
OUT: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v5"
P279: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/derived.P279.tsv.gz"
P279STAR: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/derived.P279star.tsv.gz"
PROPERTY_DATATYPES: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/metadata.property.datatypes.tsv.gz"
QUALIFIERS: "/Users/pedroszekely/Downloads/kypher/wikidata_os_v1/qualifiers.tsv.gz"
QUALIFIERS_TIME: "/Users

In [4]:
%cd {output_path}

/Users/pedroszekely/Downloads/kypher


In [5]:
!mkdir {output_folder}
!mkdir {temp_folder}

mkdir: wikidata_os_v5: File exists
mkdir: temp.wikidata_os_v5: File exists


# Wikidata in KGTK
KGTK has the ability to import a Wikidata JSON dump and covert it to the KGTK representation to make it easy to process the full Wikidata KG in a laptop. There are 86 files which include all the information available in the Wikidata dump and files containing commonly used information derived from the dump. We partitioned the files because in most use cases you only need to use a subset of the files.

The files are very large. `claims.tsv` (23GB compressed) contains all the statements in the Wikidata dump, `qualifiers.tsv` contains the qualifiers of those edges, and `labels.en.tsv`, `aliases.en.tsv` and `descriptions.en.tsv` contain the English labels, aliases and descriptions.

In [6]:
!ls -lh "$CLAIMS" "$QUALIFIERS" "$LABEL" "$ALIAS" "$DESCRIPTION"

-rw-r--r--  1 pedroszekely  staff    68M Nov 16 08:07 /Users/pedroszekely/Downloads/kypher/wikidata_os_v1/aliases.en.tsv.gz
-rw-r--r--  1 pedroszekely  staff   4.7G Nov 16 08:05 /Users/pedroszekely/Downloads/kypher/wikidata_os_v1/claims.tsv.gz
-rw-r--r--  1 pedroszekely  staff   269M Nov 16 08:08 /Users/pedroszekely/Downloads/kypher/wikidata_os_v1/descriptions.en.tsv.gz
-rw-r--r--  1 pedroszekely  staff   376M Nov 16 08:06 /Users/pedroszekely/Downloads/kypher/wikidata_os_v1/labels.en.tsv.gz
-rw-r--r--  1 pedroszekely  staff   662M Nov 16 08:43 /Users/pedroszekely/Downloads/kypher/wikidata_os_v1/qualifiers.tsv.gz


`claims.tsv` contains many edges:

In [7]:
!time zcat < "$CLAIMS" | wc

 254135077 1578463882 20285305033

real	1m13.751s
user	2m5.065s
sys	0m7.557s


# KGTK Data Model
The KGTK data model is a generalization of RDF and property graphs, inspired by the Wikidata data model. In KGTK, a KG is represented using TSV files with four columns: three columns to store the subject, predicate and object of a triple, and a fourth column to store an identifier for the triple. By convention, we use the heading `id` for the identifier, `node1` for the subject, `node2` for the object and `label` for the predicate, as it labels the edge between `node1` and `node2`. The order of the columns is arbitrary.

All KGTK files must include the required `id`, `node1`, `label` and `node2` columns, and can contain additional columns to store addtional information about an edge or the nodes in the edge. We will explain the details after we discuss *qualifiers*.
Let's take a look at the first few lines of the `claims.tsv` file. We see the four required columns and two additional columns that the Wikidata import includes to facilitate processing of the `claims` file using custom scripts. The `rank` column records the Wikidata rank of a statement, and the `node2;wikidatatype` records the Wikidata type of the value in the `node2` column.

## Claims

In [8]:
!zcat < "$CLAIMS" | head | column -t -s $'\t'

zcat: error writing to output: Broken pipe
id                              node1  label  node2                                    rank    node2;wikidatatype
P10-P1628-32b85d-7927ece6-0     P10    P1628  "http://www.w3.org/2006/vcard/ns#Video"  normal  url
P10-P1628-acf60d-b8950832-0     P10    P1628  "https://schema.org/video"               normal  url
P10-P1629-Q34508-bcc39400-0     P10    P1629  Q34508                                   normal  wikibase-item
P10-P1659-P1651-c4068028-0      P10    P1659  P1651                                    normal  wikibase-property
P10-P1659-P18-5e4b9c4f-0        P10    P1659  P18                                      normal  wikibase-property
P10-P1659-P4238-d21d1ac0-0      P10    P1659  P4238                                    normal  wikibase-property
P10-P1659-P51-86aca4c5-0        P10    P1659  P51                                      normal  wikibase-property
P10-P1855-Q15075950-7eff6d65-0  P10    P1855  Q15075950                             

Wikidata uses numbers to identify items and properties. We can use the `wd` utility (https://github.com/maxlath/wikibase-cli) to understand the first few lines. The second line states that the `P10` property in Wikidata has an equivalent property in another ontology. Notice that each edge has a distinct id. These ids are unique identifiers for statements (the format of the id can be arbitrary, but we assigned ids so that sorting files by id arranges the information so that all edges about a subject are consecutive.

In [9]:
!wd u P10 P1628 P1629

id P10
Label video
Description relevant video. For images, use the property P18. For film trailers, qualify with "object has role" (P3831)="trailer" (Q622550)
instance of (P31): Wikidata property to link to Commons (Q18610173)

id P1628
Label equivalent property
Description equivalent property in other ontologies (use in statements on properties, use property URI)
instance of (P31): Wikidata metaproperty for ontology mapping (Q42842547)

id P1629
Label subject item of this property
Description relationship represented by the property
instance of (P31): Wikidata property for property documentation (Q19820110)


Let's look at a more meaningful example. `Q31` (https://www.wikidata.org/wiki/Q31) is the Wikidata item about Belgium. We will use the KGTK query to fetch edges about Belgium. `$kypher` is a shortcut to the `kgtk query` command where in addition we pass in the location of the SQLite database we are using ot store the files. KGTK queries use Cypher syntax (https://neo4j.com/developer/cypher/): the following simple query retrieves 10 edges where `node1` is `Q31`, the q-node for Belgium. The results include an edge with `label` `P1036` (Dewey Decimal Classification) and several edges with label `P1081` (human development index).

In [10]:
!$kypher -i "$CLAIMS" \
--match '(:Q31)-[]-()' \
--limit 10 \
| column -t -s $'\t'

        2.26 real         0.64 user         0.21 sys
id                           node1  label  node2     rank    node2;wikidatatype
Q31-P1036-c4e1ad-df86eeb8-0  Q31    P1036  "2--493"  normal  external-id
Q31-P1081-02c2ed-033524b0-0  Q31    P1081  +0.866    normal  quantity
Q31-P1081-02c2ed-7971505b-0  Q31    P1081  +0.866    normal  quantity
Q31-P1081-068470-c1c63b8d-0  Q31    P1081  +0.889    normal  quantity
Q31-P1081-068470-ddac01e0-0  Q31    P1081  +0.889    normal  quantity
Q31-P1081-144738-c1851cdc-0  Q31    P1081  +0.905    normal  quantity
Q31-P1081-175742-c07ac1c8-0  Q31    P1081  +0.888    normal  quantity
Q31-P1081-19636d-c08dd8a8-0  Q31    P1081  +0.896    normal  quantity
Q31-P1081-1efc03-433a7a4d-0  Q31    P1081  +0.913    normal  quantity
Q31-P1081-1f8602-ddac530d-0  Q31    P1081  +0.852    normal  quantity


The output of the command above is hard to read because we are seeing the numeric Wikidata identifiers. To make the output more readable, we need to look up the labels of the Wikidata nodes. This information is in the `labels.en.tsv` file.

In [11]:
!zcat < "$LABEL" | head | column -t -s $'\t'

zcat: id              node1  label  node2
error writing to outputP10-label-en    P10    label  'video'@en
: P1000-label-en  P1000  label  'record held'@en
Broken pipe
P1001-label-en  P1001  label  'applies to jurisdiction'@en
P1002-label-en  P1002  label  'engine configuration'@en
P1003-label-en  P1003  label  'National Library of Romania ID'@en
P1004-label-en  P1004  label  'MusicBrainz place ID'@en
P1005-label-en  P1005  label  'Portuguese National Library ID'@en
P1006-label-en  P1006  label  'Nationale Thesaurus voor Auteurs ID'@en
P1007-label-en  P1007  label  'Lattes Platform number'@en


With KGTK accepts multiple files as input, and can do a join to retrieve the label for each property. When using multiple files, it is necessary to tag each clause with the file that provides the data for the clause. For example, the first clause is tagged with `claim` as the word `claim` is part of the file name. The variable property is used to connect the two clauses.

In [12]:
!$kypher -i "$CLAIMS" -i "$LABEL" \
--match 'claim: (n1:Q31)-[l {label: property}]-(n2), label: (property)-[:label]->(property_label)' \
--return 'l as id, n1 as node1, property as label, n2 as node2, property_label as `label;label`' \
--limit 10 \
| column -t -s $'\t'

        0.69 real         0.55 user         0.12 sys
id                           node1  label  node2     label;label
Q31-P1036-c4e1ad-df86eeb8-0  Q31    P1036  "2--493"  'Dewey Decimal Classification'@en
Q31-P1081-02c2ed-033524b0-0  Q31    P1081  +0.866    'Human Development Index'@en
Q31-P1081-02c2ed-7971505b-0  Q31    P1081  +0.866    'Human Development Index'@en
Q31-P1081-068470-c1c63b8d-0  Q31    P1081  +0.889    'Human Development Index'@en
Q31-P1081-068470-ddac01e0-0  Q31    P1081  +0.889    'Human Development Index'@en
Q31-P1081-144738-c1851cdc-0  Q31    P1081  +0.905    'Human Development Index'@en
Q31-P1081-175742-c07ac1c8-0  Q31    P1081  +0.888    'Human Development Index'@en
Q31-P1081-19636d-c08dd8a8-0  Q31    P1081  +0.896    'Human Development Index'@en
Q31-P1081-1efc03-433a7a4d-0  Q31    P1081  +0.913    'Human Development Index'@en
Q31-P1081-1f8602-ddac530d-0  Q31    P1081  +0.852    'Human Development Index'@en


Let's look at a the heads of state of Belgium recorded in property `P35`

In [13]:
!$kypher -i "$CLAIMS" -i "$LABEL" \
--match 'claims: (n1:Q31)-[l:P35]->(n2), labels: (n2)-[:label]->(n2_label)' \
--return 'l as id, n1 as node1, l.label as label, n2 as node2, n2_label as `node2;label`' \
--limit 10 \
| column -t -s $'\t'

        0.68 real         0.54 user         0.12 sys
id                            node1  label  node2      node2;label
Q31-P35-Q1079522-c82ed584-0   Q31    P35    Q1079522   'Erasme Louis Surlet de Chokier'@en
Q31-P35-Q12967-f2b9aaf3-0     Q31    P35    Q12967     'Leopold II of Belgium'@en
Q31-P35-Q12971-2088471b-0     Q31    P35    Q12971     'Leopold I of Belgium'@en
Q31-P35-Q12973-31c1b700-0     Q31    P35    Q12973     'Leopold III of Belgium'@en
Q31-P35-Q12976-f3e8a567-0     Q31    P35    Q12976     'Baudouin I of Belgium'@en
Q31-P35-Q155004-619ba603-0    Q31    P35    Q155004    'Philippe I of Belgium'@en
Q31-P35-Q3911-137f01fe-0      Q31    P35    Q3911      'Albert II of Belgium'@en
Q31-P35-Q445553-7599749f-0    Q31    P35    Q445553    'Prince Charles, Count of Flanders'@en
Q31-P35-Q55008046-725dce40-0  Q31    P35    Q55008046  'Albert I of Belgium'@en


## Qualifiers
Qualifiers provide additional information about the claims stated in the edges. For `P1081` the qualifiers tell use the year, and for head of state the qualifiers provide information about the period of time and position held by the head of state. The qualifiers can be retrieved using the identifiers of the edges. Let's retrieve the qualifiers associated with the edge for the first head of state (Erasme Louis). To do so, we use the identifier of the edge (`Q31-P35-Q1079522-c82ed584-0`) as `node1` in the `qualifiers.tsv` file. We get three edges, meaning that the edge `Q31/P35/Q1079522` has three qualifiers. Note that the qualifier edges are the same as any other edge in KGTK, having `id`, `node1`, `label` and `node2` columns:

In [14]:
!$kypher -i "$QUALIFIERS" \
--match '(n1:`Q31-P35-Q1079522-c82ed584-0`)-[l]->(n2)' \
--limit 10 \
| column -t -s $'\t'

        0.66 real         0.53 user         0.12 sys
id                                         node1                        label  node2                     node2;wikidatatype
Q31-P35-Q1079522-c82ed584-0-P39-Q477406-0  Q31-P35-Q1079522-c82ed584-0  P39    Q477406                   wikibase-item
Q31-P35-Q1079522-c82ed584-0-P580-106076-0  Q31-P35-Q1079522-c82ed584-0  P580   ^1831-02-25T00:00:00Z/11  time
Q31-P35-Q1079522-c82ed584-0-P582-774519-0  Q31-P35-Q1079522-c82ed584-0  P582   ^1831-07-20T00:00:00Z/11  time


Let's make them readable: the following query combines the patterns of the previous two queries to retrieve the labels of the property and node2. The query omits the identifier of the qualifier edges to save space. Also, the headers of the two additional columns can be arbitrary, i.e., you can name them whatever you want; the names used follow a KGTK convention that enabled KGTK to automatically parse the output, which is useful if we want to use the output as an input to another KGTK command. The word before the `;` refers to one of the standard columns, and the name after the `;` refers to a property of that element. In this example, we used `label` as the column contains the label of the entity.

In [15]:
!$kypher -i "$QUALIFIERS" -i "$LABEL" \
--match 'qual: (n1:`Q31-P35-Q1079522-c82ed584-0`)-[l {label: property}]->(n2), labels: (property)-[:label]->(property_label)' \
--return 'n1 as node1, property as label, n2 as node2, property_label as `label;label`' \
--limit 10 \
| column -t -s $'\t'

        0.69 real         0.55 user         0.12 sys
node1                        label  node2                     label;label
Q31-P35-Q1079522-c82ed584-0  P39    Q477406                   'position held'@en
Q31-P35-Q1079522-c82ed584-0  P580   ^1831-02-25T00:00:00Z/11  'start time'@en
Q31-P35-Q1079522-c82ed584-0  P582   ^1831-07-20T00:00:00Z/11  'end time'@en


Let's put all the values of `P35` in a file, which we will conveniently name `Q31.P35.tsv`

In [16]:
!$kypher -i "$CLAIMS" \
--match '(n1:Q31)-[l:P35]->(n2)' \
--return 'l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q31.P35.tsv

        0.64 real         0.51 user         0.11 sys


Now we are going to combine the `P35` edges of Belgium with the qualifiers. To do this we will run a query that uses the edges that we stored in `Q31.P35.tsv`, and retrieve the qualifiers for each of those edges; the result of our query will be the qualifier edges of the head of state edges. To union the qualifier edges with the claim edges, we feed the output of the query to the `cat` command (concatenate), and then feed the output to the `sort2` command to sort the edges. The first 12 edges are shown below. We see a claim edge followed by the qualifiers defined for it.

This snippet illustrates that KGTK commands can be chained using the `/` chain operator to compose more complex workflows.

In [17]:
!$kypher -i "$QUALIFIERS" -i "$TEMP"/Q31.P35.tsv \
--match 'P35: ()-[l]->(), qual: (l)-[lq]->(n2)' \
--return 'lq as id, l as node1, lq.label as label, n2 as node2' \
/ cat -i - -i "$TEMP"/Q31.P35.tsv \
/ sort2 \
| head -12 \
| column -t -s $'\t'

id                                         node1                        label  node2
Q31-P35-Q1079522-c82ed584-0                Q31                          P35    Q1079522
Q31-P35-Q1079522-c82ed584-0-P39-Q477406-0  Q31-P35-Q1079522-c82ed584-0  P39    Q477406
Q31-P35-Q1079522-c82ed584-0-P580-106076-0  Q31-P35-Q1079522-c82ed584-0  P580   ^1831-02-25T00:00:00Z/11
Q31-P35-Q1079522-c82ed584-0-P582-774519-0  Q31-P35-Q1079522-c82ed584-0  P582   ^1831-07-20T00:00:00Z/11
Q31-P35-Q12967-f2b9aaf3-0                  Q31                          P35    Q12967
Q31-P35-Q12967-f2b9aaf3-0-P39-Q13592862-0  Q31-P35-Q12967-f2b9aaf3-0    P39    Q13592862
Q31-P35-Q12967-f2b9aaf3-0-P580-f29037-0    Q31-P35-Q12967-f2b9aaf3-0    P580   ^1865-12-17T00:00:00Z/11
Q31-P35-Q12967-f2b9aaf3-0-P582-136f02-0    Q31-P35-Q12967-f2b9aaf3-0    P582   ^1909-12-17T00:00:00Z/11
Q31-P35-Q12971-2088471b-0                  Q31                          P35    Q12971
Q31-P35-Q12971-2088471b-0-P39-Q13592862-0  Q31-P35-Q12971-20884

## Summary

- KGTK represents graphs in TSV files with standard columns `id`, `node1`, `label` and `node2`
- It is possible to include arbitrary additional columns in KGTK files
- The identifier of an edge can be used as a node in another edge enabling the representation of edges about edges
- KGTK provides a powerful query command based on Cypher as well as a host of other commands, type `kgtk --help` to see the list of commands.

# Use Case: A Knowledge Graph About Alocholic Beverages
We are going to build a small KG about alcoholoc beverages by extracting from Wikidata the subgraph that relates to alcoholic beverages (https://www.wikidata.org/wiki/Q154)

### Step 1: create a list of all descendants of `alcoholic beverage` (https://www.wikidata.org/wiki/Q154)

In [18]:
!wd u Q154

id Q154
Label alcoholic beverage
Description drink containing alcohols, typically ethanol
instance of (P31):  drug (Q8386) | carcinogen (Q187661)
subclass of (P279): drink (Q40050)


Wikidata uses two properties to organize entities in a hierarchy: the `instance of` property (`P31`) and the `subclass of` (`P279`) property. In many cases, the distinction between instance of and subclass of is subtle, and we find many situations in Wikidata where either one or the other is used to organize hierarchies. For this reason, we created a new property called `isa` that contains the union of `P31` and `P279` and stored in the file `derived.isa.tsv`

In [19]:
!zcat < "$ISA" | head -5

node1	label	node2
P10	isa	Q18610173
P1000	isa	Q18608871
P1001	isa	Q15720608
P1001	isa	Q22984026
zcat: error writing to output: Broken pipe


To get all the alcoholic beverages, we need to get all entities that are `isa` of alcoholic beverage (`Q154`) or that are `isa` of any descendant of `Q154` in the `subclass of` (`P279`) hierarchy. The length of the chain of `P279` edges can be arbitrarily long. To support this uise case, KGTK offers the `derived.P279star.tsv` file that contains edges `n1/P279star/n2` if `n1` is a descendant of `n2` on chains of `P279` edges, includiing chains of zero length (`n1/P279star/n1`).

In [20]:
!zcat < "$P279STAR" | head -5 | column -t -s $'\t'

zcat: node1     label     node2     id
Q1000032  P279star  Q1000032  Q1000032-P279star-Q1000032-0000
Q1000032  P279star  Q1150070  Q1000032-P279star-Q1150070-0000
Q1000032  P279star  Q1190554  Q1000032-P279star-Q1190554-0000
error writing to outputQ1000032  P279star  Q133500   Q1000032-P279star-Q133500-0000
: Broken pipe


To get all alcoholic beverages, we need to find all nodes `n1` that are connected to `Q154` with an `isa` edge and a chain of `P279` edges:

In [21]:
!$kypher -i "$ISA" -i "$P279STAR" -i "$LABEL" \
--match 'isa: (n1)-[]->(n2), star: (n2)-[]->(n3:Q154), label: (n1)-[]->(n1l)' \
--return 'n1 as node1, n1l as `node1;label`, n3 as node2, "isastar" as label' \
-o "$TEMP"/Q154.descendant.tsv

        2.94 real         0.66 user         0.63 sys


Here is a sample of alcoholic beverages in Wikidata

In [22]:
!head "$TEMP"/Q154.descendant.tsv | column -t -s $'\t'

node1      node1;label                  node2  label
Q1350656   'Corn whiskey'@en            Q154   isastar
Q20713240  'Buckwheat whisky'@en        Q154   isastar
Q2535077   'Rye Whiskey'@en             Q154   isastar
Q536976    'Canadian whisky'@en         Q154   isastar
Q7991845   'Wheat whiskey'@en           Q154   isastar
Q10429117  'Beyaz'@en                   Q154   isastar
Q1069954   'Prosecco'@en                Q154   isastar
Q1094850   'Clairette du Languedoc'@en  Q154   isastar
Q1135592   'Cortese di Gavi'@en         Q154   isastar


An the total number:

In [23]:
!wc "$TEMP"/Q154.descendant.tsv

    3251   16116  133341 /Users/pedroszekely/Downloads/kypher/temp.wikidata_os_v5/Q154.descendant.tsv


The computation of `Q154.descendant.tsv` can be implemented in SPARQL using the common `P31/P279*` graph pattern, but the query will time out if the result size is large. For example, the query will time out when requesting all descendants of chemical compounds, as there are over one million chemical compounds in Wikidata. The query can be easily done in KGTK.

### Step 2: get the incoming and outgoing edges
We want out graph to have the neighbors of all alcoholic beverages, so we need to get the incoming and outgoing edges.

The following query gets the outgoing edges.

In [24]:
!$kypher -i "$CLAIMS" -i "$TEMP"/Q154.descendant.tsv \
--match 'Q154: (n1)-[]->(), claims: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.node1.tsv.gz

        2.21 real         0.81 user         0.40 sys


We see that we are getting several properties for our items:

In [25]:
!zcat < "$TEMP"/Q154.node1.tsv.gz | head | column -t -s $'\t'

id                                   node1     label  node2
Q1000737-P1435-Q17297633-53903946-0  Q1000737  P1435  Q17297633
Q1000737-P1454-Q460178-8ad4931b-0    Q1000737  P1454  Q460178
Q1000737-P159-Q16003-31e24011-0      Q1000737  P159   Q16003
Q1000737-P17-Q183-24107fe2-0         Q1000737  P17    Q183
Q1000737-P18-147fc9-667304f8-0       Q1000737  P18    "Marthabräuhalle 2011-04-03.jpg"
Q1000737-P31-Q131734-f97bd6f6-0      Q1000737  P31    Q131734
Q1000737-P31-Q15075508-a4c83928-0    Q1000737  P31    Q15075508
Q1000737-P373-689157-3110aade-0      Q1000737  P373   "Marthabräu"
Q1000737-P452-Q869095-f5d8e7a2-0     Q1000737  P452   Q869095
zcat: error writing to output: Broken pipe


Now get the incoming edges:

In [26]:
!$kypher -i "$CLAIMS" -i "$TEMP"/Q154.descendant.tsv \
--match 'Q154: (n1)-[]->(), claims: (n3)-[l]->(n1)' \
--return 'distinct l as id, n3 as node1, l.label as label, n1 as node2' \
-o "$TEMP"/Q154.node2.tsv.gz

        2.02 real         0.74 user         0.39 sys


Here is a sample of the edges we are getting

In [27]:
!zcat < "$TEMP"/Q154.node2.tsv.gz | head | column -t -s $'\t'

id                                  node1      label  node2
Q1350656-P279-Q1007164-7e3ecba9-0   Q1350656   P279   Q1007164
Q20713240-P279-Q1007164-b3112260-0  Q20713240  P279   Q1007164
Q2535077-P279-Q1007164-b2d3684b-0   Q2535077   P279   Q1007164
zcat: Q536976-P279-Q1007164-8bf7467b-0    Q536976    P279   Q1007164
Q7991845-P279-Q1007164-18bc383a-0   Q7991845   P279   Q1007164
Q10337004-P186-Q10210-c56dd7ce-0    Q10337004  P186   Q10210
Q10429117-P31-Q10210-d342f061-0     Q10429117  P31    Q10210
Q1051699-P279-Q10210-65d32c67-0     Q1051699   P279   Q10210
Q1058259-P279-Q10210-e204554a-0     Q1058259   P279   Q10210
error writing to output: Broken pipe


Concatenate the incoming and outgoing edges to put them in a single file:

In [28]:
!$kgtk cat -i "$TEMP"/Q154.node1.tsv.gz -i "$TEMP"/Q154.node2.tsv.gz -o "$TEMP"/Q154.claims.tsv.gz

        0.99 real         0.86 user         0.12 sys


We have over 30,000 edges:

In [29]:
!zcat < "$TEMP"/Q154.claims.tsv.gz | wc

   28142  116045 1584824


Summary of where we are:
- Computed the list of entities below alcoholic beverage
- Found all incoming and outgoing edges to these entities; for the new entities we bring in, we have no information, we only have the q-node

Not having any information about the entities connected to the alcoholic beverages is limiting, so let's get their outgoing edges. We run the query with `Q154.claims.tsv` which will use all the entities in our graph, including the alcoholic beverages for which we already got outgoing edges; no harm done, as we can eliminate duplicated later.

In [30]:
!$kypher -i "$CLAIMS" -i "$TEMP"/Q154.claims.tsv.gz \
--match 'Q154: ()-[]->(n1), claims: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.hop.out.tsv.gz

        5.88 real         3.79 user         0.66 sys


For sanity check, let's take a peek:

In [31]:
!zcat < "$TEMP"/Q154.hop.out.tsv.gz | head | column -t -s $'\t'

id                             node1  label  node2
Q1000-P1036-9bef62-f77ac5cf-0  Q1000  P1036  "2--6721"
Q1000-P1081-0d345f-3a33abf5-0  Q1000  P1081  +0.641
zcat: Q1000-P1081-0d345f-6da37c02-0  Q1000  P1081  +0.641
Q1000-P1081-1100e3-c7631769-0  Q1000  P1081  +0.624
Q1000-P1081-1ada51-7c71c229-0  Q1000  P1081  +0.639
Q1000-P1081-345681-88a99cab-0  Q1000  P1081  +0.702
Q1000-P1081-347db1-da0e5e03-0  Q1000  P1081  +0.637
Q1000-P1081-419245-b03a8b59-0  Q1000  P1081  +0.647
Q1000-P1081-419245-f8cd58e8-0  Q1000  P1081  +0.647
error writing to output: Broken pipe


Let's consolidate our edge files into one larger file. We use compact to remove duplicates and sort to keep edges for the same subject together:

In [32]:
!$kgtk cat -i "$TEMP"/Q154.claims.tsv.gz -i "$TEMP"/Q154.hop.out.tsv.gz \
/ compact \
/ sort2 \
-o "$TEMP"/Q154.edges.1.tsv.gz

        4.61 real         6.38 user         0.65 sys


Now we have over 170,000 edges:

In [33]:
!zcat < "$TEMP"/Q154.edges.1.tsv.gz | wc

  165133  678398 8868474


Take a peek:

In [34]:
!zcat < "$TEMP"/Q154.edges.1.tsv.gz | head | column -t -s $'\t'

id                                node1  label  node2
P1389-P1855-Q1109662-9e2ef218-0   P1389  P1855  Q1109662
P1582-P1855-Q17329207-f4ef508d-0  P1582  P1855  Q17329207
P2581-P1855-Q7639844-08b3a4c7-0   P2581  P1855  Q7639844
P2665-P1855-Q1067702-402a80a9-0   P2665  P1855  Q1067702
P2665-P1855-Q170210-30d44f0b-0    P2665  P1855  Q170210
P5420-P1855-Q44-209cffb1-0        P5420  P1855  Q44
P5420-P1855-Q722338-73d7be75-0    P5420  P1855  Q722338
P6088-P1855-Q1543214-3d934541-0   P6088  P1855  Q1543214
P6088-P1855-Q4626-4ed65964-0      P6088  P1855  Q4626
zcat: error writing to output: Broken pipe


Once we have all the alcoholic beverages, we want to get the upper ontology of all the classes used, so that every class in our KG has a path to the root of the ontology. For example, first go to `drink` (`Q40050`), then to `liquid` (`Q11435`), then `fluid` (`Q102205`) and so on until we reach `entity` (`Q35120`).

To do this, we need to get all the `isa` of all items in our graph, then get `P279star` so we get the list of all classes that these items descend from. Finally we need to get all the `P279` edges between them.

In [35]:
!$kypher -i "$TEMP"/Q154.edges.1.tsv.gz -i "$P279STAR" -i "$ISA" \
--match 'Q154: (n1)-[]->(), isa: (n1)-[]->(n2), P279: (n2)-[]->(class)' \
--return 'distinct class as node1' \
-o "$TEMP"/Q154.classes.tsv

       14.52 real         9.08 user         1.59 sys


We have almost 3,000 classes in the upper ontology for the entities in our graph:

In [36]:
!wc "$TEMP"/Q154.classes.tsv

    2846    2846   24939 /Users/pedroszekely/Downloads/kypher/temp.wikidata_os_v5/Q154.classes.tsv


Now use the `derived.P279.tsv` file to get the `P279` edges that connect a class to its superclass.

In [37]:
!$kypher -i "$TEMP"/Q154.classes.tsv -i "$P279" \
--match 'Q154: (class)-[]->(), P279: (class)-[l]->(super)' \
--return 'distinct l as id, class as node1, l.label as label, super as node2' \
-o "$TEMP"/Q154.P279.tsv

        1.22 real         0.59 user         0.24 sys


We get close to 5,000 `P279` edges in the upper ontology; we will take care of potential duplicates at a final cleanup step:

In [38]:
!wc "$TEMP"/Q154.P279.tsv

    4517   18068  249492 /Users/pedroszekely/Downloads/kypher/temp.wikidata_os_v5/Q154.P279.tsv


We see several q-nodes below `entity` (`Q35120`), a good indication that we computed the upper ontology correctly:

In [39]:
!grep Q35120 "$TEMP"/Q154.P279.tsv | head -5 | column -t -s $'\t'

Q16686448-P279-Q35120-674edbf9-0  Q16686448  P279  Q35120
Q35120-P279-25b964-0520e300-0     Q35120     P279  novalue
Q58415929-P279-Q35120-75659d0c-0  Q58415929  P279  Q35120
Q23958946-P279-Q35120-70a9ed90-0  Q23958946  P279  Q35120
Q488383-P279-Q35120-5fad2ad7-0    Q488383    P279  Q35120


Let's consolidate the edges again:

In [40]:
!$kgtk cat -i "$TEMP"/Q154.edges.1.tsv.gz -i "$TEMP"/Q154.P279.tsv \
/ compact \
/ sort2 \
-o "$TEMP"/Q154.edges.2.tsv.gz

        4.40 real         5.99 user         0.63 sys


We have over 175,000 edges:

In [41]:
!zcat < "$TEMP"/Q154.edges.2.tsv.gz | wc

  169047  694054 9085731


Summary:
- We have the instances of alcoholic beverages
- We added incoming and outgoing edges
- For the outgoing edges, we went one hop forward
- We got the upper ontology

The properties are also items in Wikidata, so let's collect them all and get their edges.

In [42]:
!$kypher -i "$TEMP"/Q154.edges.2.tsv.gz \
--match '()-[l {label: property}]->()' \
--return 'distinct property as node1' \
-o "$TEMP"/Q154.properties.tsv

        1.65 real         1.82 user         0.27 sys


In [43]:
!head "$TEMP"/Q154.properties.tsv | column -t -s $'\t'

node1
P10
P1001
P1003
P1004
P1005
P1006
P101
P1014
P1015


Let's get the edges of these properties:

In [44]:
!$kypher -i "$CLAIMS" -i "$TEMP"/Q154.properties.tsv \
--match 'Q154: (p)-[]->(), claims: (p)-[l]->(n2)' \
--return 'distinct l as id, p as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.properties.edges.tsv

        1.09 real         0.71 user         0.20 sys


Take a peek, looks like what we had before as the file is sorted, let's proceed:

In [45]:
!head "$TEMP"/Q154.properties.edges.tsv | column -t -s $'\t'

id                              node1  label  node2
P10-P1628-32b85d-7927ece6-0     P10    P1628  "http://www.w3.org/2006/vcard/ns#Video"
P10-P1628-acf60d-b8950832-0     P10    P1628  "https://schema.org/video"
P10-P1629-Q34508-bcc39400-0     P10    P1629  Q34508
P10-P1659-P1651-c4068028-0      P10    P1659  P1651
P10-P1659-P18-5e4b9c4f-0        P10    P1659  P18
P10-P1659-P4238-d21d1ac0-0      P10    P1659  P4238
P10-P1659-P51-86aca4c5-0        P10    P1659  P51
P10-P1855-Q15075950-7eff6d65-0  P10    P1855  Q15075950
P10-P1855-Q69063653-c8cdb04c-0  P10    P1855  Q69063653


Let's consolidate the edges again:

In [46]:
!$kgtk cat -i "$TEMP"/Q154.edges.2.tsv.gz -i "$TEMP"/Q154.properties.edges.tsv \
/ compact \
/ sort2 \
-o "$TEMP"/Q154.edges.3.tsv.gz

        4.89 real         6.57 user         0.61 sys


The number of edges grew a bit to 206,000

In [47]:
!zcat < "$TEMP"/Q154.edges.3.tsv.gz | wc

  197521  811687 10791930


Summary:
- We have the instances of alcoholic beverages
- We added incoming and outgoing edges
- For the outgoing edges, we went one hop forward
- We got the upper ontology
- And we have the edges on all the properties being used

We will stop adding nodes to the KG at this time, and proceed to add the labels for all the nodes.

### Step 3: get the labels, aliases and descriptions of all the items in our KG
Before we start, let's define an environment variable to hold the final edges file so that if we change our mind later, we can update it without having to change the commands below.

In [48]:
os.environ["Q154GRAPH"] = os.environ["TEMP"] + "/Q154.edges.3.tsv.gz"

In [49]:
!ls "$Q154GRAPH"

/Users/pedroszekely/Downloads/kypher/temp.wikidata_os_v5/Q154.edges.3.tsv.gz


Get the labels of the `node1` nodes

In [50]:
!$kypher -i "$Q154GRAPH" -i "$LABEL" \
--match 'Q154: (n1)-[]-(), label: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.label.node1.tsv.gz

        4.38 real         2.49 user         0.84 sys


In [51]:
!zcat < "$TEMP"/Q154.label.node1.tsv.gz | head | column -t -s $'\t'

zcat: error writing to output: Broken pipe
id              node1  label  node2
P10-label-en    P10    label  'video'@en
P1001-label-en  P1001  label  'applies to jurisdiction'@en
P1003-label-en  P1003  label  'National Library of Romania ID'@en
P1004-label-en  P1004  label  'MusicBrainz place ID'@en
P1005-label-en  P1005  label  'Portuguese National Library ID'@en
P1006-label-en  P1006  label  'Nationale Thesaurus voor Auteurs ID'@en
P101-label-en   P101   label  'field of work'@en
P1014-label-en  P1014  label  'Getty AAT ID'@en
P1015-label-en  P1015  label  'NORAF ID'@en


Get the labels of the `node2` nodes

In [52]:
!$kypher -i "$Q154GRAPH" -i "$LABEL" \
--match 'Q154: ()-[]-(n2), label: (n2)-[l]->(n2)' \
--return 'distinct l as id, n2 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.label.node2.tsv.gz

        6.99 real         1.05 user         1.70 sys


Concatenate the two label files

In [53]:
!$kgtk cat -i "$TEMP"/Q154.label.node1.tsv.gz -i "$TEMP"/Q154.label.node2.tsv.gz \
-o "$TEMP"/labels.tsv.gz

        0.77 real         0.64 user         0.11 sys


Get the aliases of `node1` nodes

In [54]:
!$kypher -i "$Q154GRAPH" -i "$ALIAS" \
--match 'Q154: (n1)-[]-(), alias: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.alias.node1.tsv.gz

        2.52 real         1.26 user         0.46 sys


Get the aliases of `node2` nodes

In [55]:
!$kypher -i "$Q154GRAPH" -i "$ALIAS" \
--match 'Q154: ()-[]-(n2), alias: (n2)-[l]->(n2)' \
--return 'distinct l as id, n2 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.alias.node2.tsv.gz

        1.04 real         0.63 user         0.19 sys


Concatenate the two alias files

In [57]:
!$kgtk cat -i "$TEMP"/Q154.alias.node1.tsv.gz -i "$TEMP"/Q154.alias.node2.tsv.gz \
-o "$TEMP"/alias.tsv.gz

        1.20 real         0.78 user         0.20 sys


Get the descriptions of `node1` nodes

In [58]:
!$kypher -i "$Q154GRAPH" -i "$DESCRIPTION" \
--match 'Q154: (n1)-[]-(), description: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.description.node1.tsv.gz

        3.52 real         1.12 user         0.69 sys


Get the descriptions of `node2` nodes

In [59]:
!$kypher -i "$Q154GRAPH" -i "$DESCRIPTION" \
--match 'Q154: ()-[]-(n2), description: (n2)-[l]->(n2)' \
--return 'distinct l as id, n2 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.description.node2.tsv.gz

        7.32 real         1.14 user         1.83 sys


Concatenate the two description files

In [60]:
!$kgtk cat -i "$TEMP"/Q154.description.node1.tsv.gz -i "$TEMP"/Q154.description.node2.tsv.gz \
-o "$TEMP"/description.tsv.gz

        0.86 real         0.69 user         0.13 sys


### Step 4: get the qualifiers

In [61]:
!$kypher -i "$Q154GRAPH" -i "$QUALIFIERS" \
--match 'Q154: ()-[l]->(), qual: (l)-[lq]->(n2)' \
--return 'lq as id, l as node1, lq.label as label, n2 as node2' \
-o "$OUT"/qualifiers.tsv.gz

        5.44 real         2.32 user         0.88 sys


In [62]:
!zcat < "$TEMP"/Q154.qualifiers.tsv.gz | head | column -t -s $'\t'

id                                                node1                           label  node2
zcat: P10-P1855-Q15075950-7eff6d65-0-P10-54b214-0       P10-P1855-Q15075950-7eff6d65-0  P10    "Smoorverliefd 12 september.webm"
P10-P1855-Q15075950-7eff6d65-0-P3831-Q622550-0    P10-P1855-Q15075950-7eff6d65-0  P3831  Q622550
P10-P1855-Q69063653-c8cdb04c-0-P10-6fb08f-0       P10-P1855-Q69063653-c8cdb04c-0  P10    "Couch Commander.webm"
error writing to output: P10-P1855-Q7378-555592a4-0-P10-8a982d-0           P10-P1855-Q7378-555592a4-0      P10    "Elephants Dream (2006).webm"
Broken pipe
P10-P2302-Q21502404-d012aef4-0-P1793-f4c2ed-0     P10-P2302-Q21502404-d012aef4-0  P1793  "(?i).+\\\\.(webm\\|ogv\\|ogg\\|gif)"
P10-P2302-Q21502404-d012aef4-0-P2316-Q21502408-0  P10-P2302-Q21502404-d012aef4-0  P2316  Q21502408
P10-P2302-Q21502404-d012aef4-0-P2916-cb0917-0     P10-P2302-Q21502404-d012aef4-0  P2916  'filename with extension: webm, ogg, ogv, or gif (case insensitive)'@en
P10-P2302-Q21510851-5224

In [63]:
!zcat < "$TEMP"/Q154.qualifiers.tsv.gz | wc

  109816  446163 10639203


### Step 5: consolidate all the files

In [64]:
!wget https://raw.githubusercontent.com/usc-isi-i2/kgtk/dev/kgtk-properties/kgtk.properties.tsv -O "$TEMP"/kgtk.properties.tsv

--2020-12-17 15:20:06--  https://raw.githubusercontent.com/usc-isi-i2/kgtk/dev/kgtk-properties/kgtk.properties.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.40.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.40.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2617 (2.6K) [text/plain]
Saving to: ‘/Users/pedroszekely/Downloads/kypher/temp.wikidata_os_v5/kgtk.properties.tsv’

/Users/pedroszekely 100%[===================>]   2.56K  --.-KB/s    in 0s      

2020-12-17 15:20:07 (18.5 MB/s) - ‘/Users/pedroszekely/Downloads/kypher/temp.wikidata_os_v5/kgtk.properties.tsv’ saved [2617/2617]



In [65]:
!head "$TEMP"/kgtk.properties.tsv | column -t -s $'\t'

node1     label        node2                                   id
isa       label        "is a"@en                               isa-label-e79b73
isa       alias        "isa"@en                                isa-alias-7773c5
isa       description  "Instance or subclass relationship"@en  isa-description-0b5cdc
isa       P31          Q18616576                               isa-P31-Q18616576
isa       P31          Q28326461                               isa-P31-Q28326461
isa       P31          Q18647519                               isa-P31-Q18647519
isa       data_type    wikibase-item                           isa-data_type-643cc9
P279star  label        "is a"@en                               P279star-label-e79b73
P279star  alias        "isa"@en                                P279star-alias-7773c5


In [66]:
!zcat < "$PROPERTY_DATATYPES" | head

id	node1	label	node2
P10-datatype	P10	datatype	commonsMedia
P1000-datatype	P1000	datatype	wikibase-item
P1001-datatype	P1001	datatype	wikibase-item
P1002-datatype	P1002	datatype	wikibase-item
P1003-datatype	P1003	datatype	external-id
P1004-datatype	P1004	datatype	external-id
P1005-datatype	P1005	datatype	external-id
P1006-datatype	P1006	datatype	external-id
P1007-datatype	P1007	datatype	external-id
zcat: error writing to output: Broken pipe


In [67]:
!$kypher -i "$Q154GRAPH" -i "$PROPERTY_DATATYPES" \
--match 'Q15: (n1)-[]->(), property: (n1)-[l:datatype]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q15.metadata.property.datatype.tsv.gz

        0.86 real         0.64 user         0.16 sys


In [76]:
!$kgtk cat \
-i "$TEMP"/labels.tsv.gz \
-i "$TEMP"/alias.tsv.gz \
-i "$TEMP"/description.tsv.gz \
-i "$TEMP"/Q154.edges.3.tsv.gz \
-i "$TEMP"/kgtk.properties.tsv \
-i "$TEMP"/Q15.metadata.property.datatype.tsv.gz \
/ compact \
/ sort2 \
-o "$OUT"/all.tsv.gz

        5.62 real         7.25 user         0.66 sys


In [77]:
!zcat < "$OUT"/all.tsv.gz | wc

  227109 1002887 12739518


## Step 6: partition the files to follow the conventions KGTK uses for Wikidata

We'll use the partition-wikidata notebook to complete this step. This notebook expects an input file that includes all edges and qualifiers together. We also need to specify a directory where partitioned files should be created, and a directory where temporary files can be sent (this should be different from our temp directory as the partition notebook will clear any existing files in this folder).

In [70]:
!mkdir $OUT/parts

In [78]:
!$kgtk cat -i $OUT/all.tsv.gz -i $OUT/qualifiers.tsv.gz -o $TEMP/all_and_qualifiers.tsv.gz

        5.16 real         4.92 user         0.17 sys


In [ ]:
pm.execute_notebook(
    os.environ["EXAMPLES_DIR"] + "/partition-wikidata.ipynb",
    os.environ["TEMP"] + "/partition-wikidata.out.ipynb",
    parameters=dict(
        wikidata_input_path = os.environ["TEMP"] + "/all_and_qualifiers.tsv.gz",
        wikidata_parts_path = os.environ["OUT"] + "/parts",
        temp_folder_path = os.environ["OUT"] + "/parts/temp",
        sort_extras = "--buffer-size 30% --temporary-directory $OUT/parts/temp",
        verbose = False
    )
)

The partition-wikidata notebook created the following partitioned kgtk-files:

In [73]:
!ls $OUT/parts

temp


# Stop here: the stuff below is Pedro's scratchpad, will be deleted later

### Cleanup

Remove `novalue` and `somevalue`